In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
    builder.\
    master("local").\
    appName('linear-regression-project').\
    getOrCreate()

In [2]:
df = spark.read.csv(
    path='D:/learn-ab/learning-PySpark/sample-data/cruise-ship-info.csv',
    inferSchema=True,
    header=True
)
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [3]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [4]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       NULL|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|     NULL|       NULL| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [5]:
df.groupBy('Cruise_line').count().orderBy('count', ascending=False).show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|  Royal_Caribbean|   23|
|         Carnival|   22|
|         Princess|   17|
| Holland_American|   14|
|        Norwegian|   13|
|            Costa|   11|
|        Celebrity|   10|
|              MSC|    8|
|              P&O|    6|
|             Star|    6|
|Regent_Seven_Seas|    5|
|        Silversea|    4|
|           Cunard|    3|
|         Seabourn|    3|
|         Windstar|    3|
|          Oceania|    3|
|          Crystal|    2|
|           Disney|    2|
|          Azamara|    2|
|           Orient|    1|
+-----------------+-----+



In [6]:
from pyspark.ml.feature import StringIndexer

In [7]:
?StringIndexer

Init signature:
StringIndexer(
    *,
    inputCol: Optional[str] = None,
    outputCol: Optional[str] = None,
    inputCols: Optional[List[str]] = None,
    outputCols: Optional[List[str]] = None,
    handleInvalid: str = 'error',
    stringOrderType: str = 'frequencyDesc',
)
Docstring:     
A label indexer that maps a string column of labels to an ML column of label indices.
If the input column is numeric, we cast it to string and index the string values.
The indices are in [0, numLabels). By default, this is ordered by label frequencies
so the most frequent label gets index 0. The ordering behavior is controlled by
setting :py:attr:`stringOrderType`. Its default value is 'frequencyDesc'.

.. versionadded:: 1.4.0

Examples
--------
>>> stringIndexer = StringIndexer(inputCol="label", outputCol="indexed",
...     stringOrderType="frequencyDesc")
>>> stringIndexer.setHandleInvalid("error")
StringIndexer...
>>> model = stringIndexer.fit(stringIndDf)
>>> model.setHandleInvalid("error")
St

In [8]:
indexer = StringIndexer(
    inputCol='Cruise_line',
    outputCol='cruise_line_num'
)
indexed = indexer.fit(df).transform(df)
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_line_num|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
?VectorAssembler

Init signature:
VectorAssembler(
    *,
    inputCols: Optional[List[str]] = None,
    outputCol: Optional[str] = None,
    handleInvalid: str = 'error',
)
Docstring:     
A feature transformer that merges multiple columns into a vector column.

.. versionadded:: 1.4.0

Examples
--------
>>> df = spark.createDataFrame([(1, 0, 3)], ["a", "b", "c"])
>>> vecAssembler = VectorAssembler(outputCol="features")
>>> vecAssembler.setInputCols(["a", "b", "c"])
VectorAssembler...
>>> vecAssembler.transform(df).head().features
DenseVector([1.0, 0.0, 3.0])
>>> vecAssembler.setParams(outputCol="freqs").transform(df).head().freqs
DenseVector([1.0, 0.0, 3.0])
>>> params = {vecAssembler.inputCols: ["b", "a"], vecAssembler.outputCol: "vector"}
>>> vecAssembler.transform(df, params).head().vector
DenseVector([0.0, 1.0])
>>> vectorAssemblerPath = temp_path + "/vector-assembler"
>>> vecAssembler.save(vectorAssemblerPath)
>>> loadedAssembler = VectorAssembler.load(vectorAssemblerPath)
>>> loadedAssembler.tra

In [11]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_line_num']

In [12]:
assembler = VectorAssembler(
    inputCols=[
        'Age',
        'Tonnage',
        'passengers',
        'length',
        'cabins',
        'passenger_density',
        'cruise_line_num'
    ],
    outputCol='features'
)
df_final = assembler.transform(indexed).\
    select('features', 'crew')
df_final.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [13]:
train_data, test_data = df_final.randomSplit([0.7, 0.3])
train_data.describe().show()
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               113|
|   mean| 7.756991150442488|
| stddev|3.1996308059780656|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               45|
|   mean|7.887555555555555|
| stddev|4.210159873640924|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [14]:
from pyspark.ml.regression import LinearRegression

In [15]:
?LinearRegression

Init signature:
LinearRegression(
    *,
    featuresCol: str = 'features',
    labelCol: str = 'label',
    predictionCol: str = 'prediction',
    maxIter: int = 100,
    regParam: float = 0.0,
    elasticNetParam: float = 0.0,
    tol: float = 1e-06,
    fitIntercept: bool = True,
    standardization: bool = True,
    solver: str = 'auto',
    weightCol: Optional[str] = None,
    aggregationDepth: int = 2,
    loss: str = 'squaredError',
    epsilon: float = 1.35,
    maxBlockSizeInMB: float = 0.0,
)
Docstring:     
Linear regression.

The learning objective is to minimize the specified loss function, with regularization.
This supports two kinds of loss:

* squaredError (a.k.a squared loss)
* huber (a hybrid of squared error for relatively small errors and absolute error for     relatively large ones, and we estimate the scale parameter from training data)

This supports multiple types of regularization:

* none (a.k.a. ordinary least squares)
* L2 (ridge regression)
* L1 (Lasso)
* L

In [16]:
lr = LinearRegression(
    featuresCol='features',
    labelCol='crew',
    predictionCol='pred'
)
lr_model = lr.fit(train_data)
test_res = lr_model.evaluate(test_data)

In [17]:
lr_model.coefficients

DenseVector([-0.0177, 0.0126, -0.1259, 0.4581, 0.7432, -0.0113, 0.0612])

In [18]:
test_res.rootMeanSquaredError

1.3992964639782035

In [19]:
test_res.r2adj

0.8656514178158731

In [20]:
df_final.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [21]:
from pyspark.sql.functions import corr

In [22]:
featureCols = [
    'Age',
    'Tonnage',
    'passengers',
    'length',
    'cabins',
    'passenger_density'
]

In [23]:
for col in featureCols:
    df.select(corr('crew', col)).show()

+-------------------+
|    corr(crew, Age)|
+-------------------+
|-0.5306565039638852|
+-------------------+

+-------------------+
|corr(crew, Tonnage)|
+-------------------+
|  0.927568811544939|
+-------------------+

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

+------------------+
|corr(crew, length)|
+------------------+
|0.8958566271016579|
+------------------+

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

+-----------------------------+
|corr(crew, passenger_density)|
+-----------------------------+
|         -0.15550928421699717|
+-----------------------------+

